In [ ]:
# training_data = pd.read_csv('https://raw.githubusercontent.com/Zaid-N-Ansari/PG-Mini-Project/refs/heads/main/Data/RossmannStoreSales/train.csv')
training_data = pd.read_csv('../Data/RossmannStoreSales/train.csv')
training_data.head()

In [ ]:
# testing_data = pd.read_csv('https://raw.githubusercontent.com/Zaid-N-Ansari/PG-Mini-Project/refs/heads/main/Data/RossmannStoreSales/test.csv')
testing_data = pd.read_csv('../Data/RossmannStoreSales/test.csv')
testing_data.head()

In [ ]:
# store_data = pd.read_csv('https://raw.githubusercontent.com/Zaid-N-Ansari/PG-Mini-Project/refs/heads/main/Data/RossmannStoreSales/store.csv')
store_data = pd.read_csv('../Data/RossmannStoreSales/store.csv')
store_data

In [ ]:
training_data = training_data.merge(store_data, how='left', on='Store')
training_data

In [ ]:
testing_data = testing_data.merge(store_data, how='left', on='Store')
testing_data

In [ ]:
training_data.info()

In [ ]:
training_data['Store'] = training_data['Store'].astype(object)
training_data['Open'] = training_data['Open'].astype(object)
training_data['Promo'] = training_data['Promo'].astype(object)
training_data['Promo2'] = training_data['Promo2'].astype(object)
training_data['SchoolHoliday'] = training_data['SchoolHoliday'].astype(object)
training_data.info()

In [ ]:
testing_data.info()

In [ ]:
testing_data['Store'] = testing_data['Store'].astype(object)
testing_data['Open'] = testing_data['Open'].astype(object)
testing_data['Promo'] = testing_data['Promo'].astype(object)
testing_data['Promo2'] = testing_data['Promo2'].astype(object)
testing_data['SchoolHoliday'] = testing_data['SchoolHoliday'].astype(object)
testing_data.info()


# Understanding Individual Column Distributions



## 1. Histograms (Numerical)



## 2. Box Plots (Numerical) - for Outlier Detection


In [ ]:
for col in numerical_columns:
    plt.figure()
    sns.boxplot(y=training_data[col].dropna())
    plt.title(f'Box Plot of {col}')
    plt.ylabel(col)
    plt.show()


## 3. Count Plots / Bar Charts (Categorical)


In [ ]:
for col in ['StateHoliday', 'SchoolHoliday', 'StoreType']:
    plt.figure()
    sns.countplot(x=training_data[col].astype(str), order=training_data[col].astype(str).value_counts().index)
    plt.title(f'Counts of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.show()


## 4. Value counts for PromoInterval (which is a string)


In [ ]:
sns.countplot(x=training_data['PromoInterval'], order=training_data['PromoInterval'].value_counts().index)
plt.title('Counts of PromoInterval')
plt.xlabel('Promo Interval')
plt.ylabel('Count')
plt.tight_layout()
plt.show()


# Exploring Relationships Between Two Variables



## 1. Scatter Plots (Numerical vs. Numerical)


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Customers', y='Sales', data=training_data)
plt.title('Sales vs. Customers')
plt.xlabel('Number of Customers')
plt.ylabel('Sales')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='CompetitionDistance', y='Sales', data=training_data.dropna(subset=['CompetitionDistance']))
plt.title('Sales vs. CompetitionDistance')
plt.xlabel('Competition Distance')
plt.ylabel('Sales')
plt.show()

plt.figure(figsize=(12, 8))
sns.scatterplot(x='Customers', y='Sales', hue='StoreType', data=training_data, alpha=0.7)
plt.title('Sales vs. Customers by Store Type')
plt.xlabel('Number of Customers')
plt.ylabel('Sales')
plt.legend(title='Store Type')
plt.show()


## 2. Correlation Heatmap


In [ ]:
plt.figure(figsize=(12, 10))
corr_matrix = training_data[numerical_columns].corr()
sns.heatmap(corr_matrix, annot=True, cmap='Accent', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Numerical Features')
plt.show()


# Time-Series Specific Visualizations (Crucial for Sales Data)



### Pre-processing for time-series plots: Aggregate daily sales


In [ ]:
copy_training_data = training_data.copy()
copy_training_data = copy_training_data[copy_training_data['Sales'] > 0]

copy_training_data['Date'] = pd.to_datetime(copy_training_data['Date'])
copy_training_data['Month'] = copy_training_data['Date'].dt.month
copy_training_data['Year'] = copy_training_data['Date'].dt.year
copy_training_data['date'] = copy_training_data['Date'].dt.day
copy_training_data['DayOfWeekName'] = copy_training_data['Date'].dt.day_name()
copy_training_data['MonthName'] = copy_training_data['Date'].dt.month_name()

sales_by_year = copy_training_data.groupby(copy_training_data['Year'])['Sales'].sum().reset_index()
sales_by_year.columns = ['Year', 'Sales']

sales_by_month = copy_training_data.groupby([copy_training_data['Year'], copy_training_data['Month']])['Sales'].sum().reset_index()
sales_by_month.columns = ['Year', 'Month', 'Sales']

sales_by_week = copy_training_data.groupby([copy_training_data['Year'], copy_training_data['Date'].dt.isocalendar().week])['Sales'].sum().reset_index()
sales_by_week.columns = ['Year', 'Week', 'Sales']


## 1. Line Plot: Total Daily Sales Over Time


In [ ]:
plt.plot(sales_by_year['Year'].astype(str), sales_by_year['Sales'], marker='o')
plt.title('Total Sales by Year')
plt.xlabel('Year')
plt.ylabel('Total Sales')
plt.grid(True)
plt.show()
sales_by_year

In [ ]:
sns.lineplot(data=sales_by_month, x='Month', y='Sales', hue='Year', palette='Set1')
plt.title('Total Sales by Month (per Year)')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.grid(True)
plt.show()
sales_by_month

In [ ]:
sns.lineplot(data=sales_by_week, x='Week', y='Sales', hue='Year', palette='tab10')
plt.title('Total Sales by Week (per Year)')
plt.xlabel('Week of Year')
plt.ylabel('Total Sales')
plt.xticks(range(1, 53, 3))
plt.grid(True)
plt.show()
sales_by_week


## 2. Seasonal / Subseries Plots


In [ ]:
sns.boxplot(x='DayOfWeekName', y='Sales', data=copy_training_data.sort_values('DayOfWeek'), order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Sales Distribution by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Sales')
plt.show()

In [ ]:
sns.boxplot(x='MonthName', y='Sales', data=copy_training_data.sort_values('Month'))
plt.title('Sales Distribution by Month of Year')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.show()

In [ ]:
sns.boxplot(x='Year', y='Sales', data=copy_training_data)
plt.title('Sales Distribution by Year')
plt.xlabel('Year')
plt.ylabel('Sales')
plt.show()


# Multi-Variate Visualizations



## 3. Sales vs. Customers by Promotion Status


In [ ]:
g_holiday = sns.FacetGrid(data=copy_training_data, col='Promo_Status', col_wrap=2, height=5, aspect=1.2, sharey=True)
g_holiday.map_dataframe(sns.scatterplot, x='Customers', y='Sales', alpha=0.8, hue='StateHoliday')
g_holiday.add_legend(title='State Holiday')
g_holiday.set_titles("Promo: {col_name}")
plt.suptitle('Sales vs. Customers by Promotion Status (Colored by State Holiday)', y=1.02)
plt.show()

# Specific Domain-Driven Visualizations

## 1. Sales/Customers during Holidays

In [ ]:
sns.barplot(x='StateHoliday', y='Sales', hue='SchoolHoliday', data=copy_training_data.astype({'StateHoliday': str, 'SchoolHoliday':str}))
plt.title('Average Sales by State and School Holiday Status')
plt.xlabel('State Holiday')
plt.ylabel('Average Sales')
plt.legend(title='School Holiday')
plt.show()

## 2. Promo2 Impact on Sales

In [ ]:
sns.boxplot(x='Promo2', y='Sales', data=copy_training_data)
plt.title('Sales Distribution for Promo2 Active vs. Not Active')
plt.xlabel('Promo2 (0: Not Active, 1: Active)')
plt.ylabel('Sales')
plt.show()

## 3. Sales over time per Store Type

In [ ]:
agg_storetype_sales = copy_training_data.groupby(['DayOfWeekName', 'StoreType'])['Sales'].sum().reset_index()
plt.figure(figsize=(16, 8))
sns.lineplot(x='DayOfWeekName', y='Sales', hue='StoreType', data=agg_storetype_sales, palette='tab10')
plt.title('Total Daily Sales Over Time by Store Type')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.legend(title='Store Type')
plt.show()

## 4. Average Sales per Store (Top 10 Stores)

In [ ]:
avg_sales_per_store = copy_training_data.groupby('Store')['Sales'].mean().nlargest(10).index
top_stores = copy_training_data[copy_training_data['Store'].isin(avg_sales_per_store) & (copy_training_data['Sales'] > 0)]
plt.figure(figsize=(14, 8))
sns.barplot(x='Store', y='Sales', data=top_stores, estimator=np.mean, order=avg_sales_per_store, palette='tab10')
plt.title('Average Sales for Top 10 Performing Stores')
plt.xlabel('Store ID')
plt.ylabel('Average Sales')
plt.show()

## 5. Sales distribution per DayOfWeek and StoreType

In [ ]:
sns.boxplot(x='DayOfWeekName', y='Sales', hue='StoreType', data=copy_training_data.sort_values('DayOfWeek'), order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], palette='tab10')
plt.title('Sales Distribution by Day of Week and Store Type')
plt.xlabel('Day of Week')
plt.ylabel('Sales')
plt.legend(title='Store Type', loc='upper right', bbox_to_anchor=(0.98, 1))
plt.show()